In [1]:
import cv2
import face_recognition
import pickle

In [2]:
# 동영상
video_file = 'opencv_data/video/son_02.mp4'
# 손흥민과 teddy 사진 데이터가 저장된 데이터 파일
encoding_file = 'encoding.pickle'
# 인식하지 못한 사람에게 부여할 결과데이터
unknown_name = 'Unknown'
# 학습 방식(cnn, hog)
model_method = 'cnn'

In [3]:
# 파일로 저장한 데이터를 읽어온다.
with open(encoding_file, 'rb') as fp :
    a1 = fp.read()
    data = pickle.loads(a1)
    
def detectAndDisplay(image) :
    # 사용할 이미지의 색상 체계를 변환한다.
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # 사진에서 얼굴 부분을 가져온다.
    boxes = face_recognition.face_locations(rgb, model=model_method)
    # boxes

    # 가져온 얼굴 부분에 해당하는 데이터들을 추출한다.
    encodings = face_recognition.face_encodings(rgb, boxes)
    # encodings

    # 검출된 사람의 이름을 담을 리스트
    names = []

    # 가져온 얼굴 데이터의 수 만큼 반복한다.
    for enc in encodings :
        # 가져온 얼굴 데이터와 손흥민과 teddy 얼굴 사진이 저장되어 있는 데이터와 
        # 비교한다.
        # 첫 번째 : 손흥민과 teddy 얼굴데이터
        # 두 번째 : 방금 가져온 얼굴 데이터
        matches = face_recognition.compare_faces(data['encodings'], enc)
        # print(matches)
        name = unknown_name

        # 매칭된 결과에 True가 포함되어 있다면...
        if True in matches :
            # True 자리에 해당하는 인덱스 번호를 추출한다.
            matchIdexs = [i for (i, b) in enumerate(matches) if b]
            # print(matchIdexs)
            # 결과를 담을 딕셔너리
            counts = {}

            # True자리에 해당하는 인덱스가 담긴 리스트의 데이터 수 만큼 반복
            for i in matchIdexs :
                # print(i)
                # i 번째 이름을 추출한다.
                name = data['names'][i]
                # print(name)
                # 이름에 해당하는 값을 1 증가시킨다.
                counts[name] = counts.get(name, 0) + 1

            # print(counts)
            # 딕셔너리에 들어 있는 값들 중에 제일 큰 값의 이름을 추출한다.
            name = max(counts, key=counts.get)
            # print(name)

        # 추출한 이름을 담는다.
        names.append(name)
        # print(names)
    
    # 추출한 얼굴이 누구인지 표시한다.
    for ((top, right, bottom, left), name) in zip(boxes, names) :
        # print(top, right, bottom, left, name)

        # 인식하지 못한 사람
        if name == unknown_name :
            color = (0, 0, 255)
            line = 1
            name = ''
        # 인식한 사람
        else :
            color = (0, 255, 0)
            line = 5

        # 얼굴 영역 표시
        cv2.rectangle(image, (left, top), (right, bottom), color, line)
        # 이름 표시
        cv2.putText(image, name, (left, top - 5), cv2.FONT_HERSHEY_PLAIN, 1, color, 1)

In [4]:
cap = cv2.VideoCapture(video_file)

while True :
    ret, image = cap.read()
    
    if image is None :
        break
        
    # 인식
    detectAndDisplay(image)
    
    cv2.imshow('video', image)

    if cv2.waitKey(1) & 0xFF == ord('q') :
        break
        
cv2.destroyAllWindows()